## Estimating the Brock and Mirman (1972) model by GMM and SMM

You can observe time series data in an economy for the following variables $(c_t, k_t, w_t, r_t)$. The first line of this file contains variable labels. These data have 100 periods, which are quarterly (25 years). Suppose you think the data are generated by a pricess similar to the Brock and Mirman (1972). A simplified set of characterizing equations of the Brock and Mirman model are the following:
\begin{align}
c_t^{-1} - \beta E[r_{t+1}(c_{t+1})^{-1}] & = 0 && \text{(1)} \\
c_t - k_{t+1} - w_t - r_t k_t & = 0 && \text{(2)} \\
w_t - (1-\alpha)e^{z_t}k_t^\alpha & = 0 && \text{(3)} \\
r_t - \alpha e^{z_t} k_t^{\alpha - 1} & = 0 && \text{(4)} \\
z_t & = \rho z_{t-1} + (1-\rho)\mu + \epsilon_t && \text{(5)} \\
\epsilon_t & \sim N(0,\sigma^2)
\end{align}
The variable $c_t$ is aggregate consumption in period $t$. $k_{t+1}$ is total household savings and investment in period $t$ for which they receive a return in the next period (this model assumes full depreciation of capital). The wage per unit of labor in period $t$ is $w_t$ adn the interest rate or rate of return on investment is $r_t$. Total factor productivity is $z_t$, which follows an AR(1) process. The rest of the symbols in the equations are parameters that must be estimated:
$$ (\alpha, \beta, \rho, \mu, \sigma) $$
The constraints on these parameters are:
$$ \alpha, \beta \in (0,1), \quad \mu,\sigma > 0, \quad \rho \in (-1,1) $$
Assume that the first observation in the data file is $t=1$. Let $k_1$ be the first observation in the data file for $k_t$. Assume that $z_0 = \mu$, so that $z_1 = \mu$. Assume that the discount factor is known to be $\beta = 0.99$.

In [1]:
# Load Modules
import numpy as np
import scipy.stats as sts
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.special as spc
import scipy.integrate as integrate
import distributions as dst

In [2]:
# Import Data
macroseries = pd.read_csv('MacroSeries.csv')

(a) Estimate $\rho$ and $\mu$ by GMM using the unconditional moment condition that $E[\epsilon_t] = 0$. use data on $r_t$ and $k_t$ and equation (4) to back out a time series for $z_t$. Then use the unconditional moment condition that $E[\epsilon_{t+1} = 0$, which also implies that $E[\epsilon_{t+1}\epsilon_{t}] = 0$ to estimate $\rho$ and $\mu$. That is, use the following 2 moment conditions to estimate $rho$ and $\mu$. Report your estimated values and the value of your minimized criterion function.
\begin{align*}
E\left[z_{t+1} - \rho z_t - (1-\rho)\mu\right] & = 0 && \text{(6)} \\
E\left[\left(z_{t+1} - \rho z_t - (1-\rho)\mu\right)z_t\right] & = 0 && \text{(7)}
\end{align*}

(b) To estimate $\alpha$ and $\beta$ by GMM, we use the unconditional moment condition from equation (1).
\begin{align*}
E\left[\beta r_{t+1} \frac{c_t}{c_{t-1}} - 1 \right] = 0 && \text{(8)} 
\end{align*}
We need both $\alpha$ and $\beta$ in the moment conditions, so we substitute in the expression for $r_{t+1}$ from (4). Use your implied series for $z_t$ from part (a) and the following two moment conditions to estimate $\alpha$ and $\beta$. Report your estimated values and the value of your minimized criterion function. 
\begin{align*}
E \left[\beta \alpha e^{z_{t+1}} k_{t+1}^{\alpha-1} \frac{c_t}{c_{t+1}} - 1 \right] = 0 && \text{(9)} \\
E \left[\left(\beta \alpha e^{z_{t+1}} k_{t+1}^{\alpha-1} \frac{c_t}{c_{t+1}} - 1\right) w_t\right] = 0 && \text{(10)} 
\end{align*}

In [3]:
# Define Equation (4)
def eq_4(r_t,k_t,alpha):
    return np.log(r_t/(alpha*k_t**(alpha-1)))

# Define Equation (6)
def eq_6(z_t,z_t_1, rho,mu):
    return z_t_1 - rho*z_t - (1-rho)*mu

# Define Equation (7)
def eq_7(z_t,z_t_1, rho,mu):
    return (z_t_1 - rho*z_t - (1-rho)*mu)*z_t

# Define Equation (8) - Unused
def eq_8(r_t_1,c_t,c_t_1,beta):
    return (beta*r_t_1*c_t/c_t_1 - 1)

# Define Equation (9)
def eq_9(z_t_1,c_t,c_t_1,k_t_1,alpha,beta):
    return beta*alpha*np.exp(z_t_1)*(k_t_1**(alpha-1))*c_t/c_t_1 - 1

# Define Equation (10)
def eq_10(z_t_1,c_t,c_t_1,k_t_1,w_t,alpha,beta):
    return (beta*alpha*np.exp(z_t_1)*(k_t_1**(alpha-1))*c_t/c_t_1 - 1)*w_t

In [4]:
def GMM_err(alpha,beta,rho,mu,c_t,k_t,w_t,r_t):
    
    z_t = eq_4(r_t,k_t,alpha)
    z_t_1 = z_t.shift(-1)
    r_t_1 = r_t.shift(-1)
    c_t_1 = c_t.shift(-1)
    k_t_1 = k_t.shift(-1)
    
    return np.array([np.nanmean(eq_6(z_t,z_t_1, rho,mu)), 
                     np.nanmean(eq_7(z_t,z_t_1, rho,mu)),
                     np.nanmean(eq_9(z_t_1,c_t,c_t_1,k_t_1,alpha,beta)),
                     np.nanmean(eq_10(z_t_1,c_t,c_t_1,k_t_1,w_t,alpha,beta))])

def GMM_criterion(params, *args):
    
    alpha,beta,rho,mu = params
    c_t,k_t,w_t,r_t,W = args
    
    err = GMM_err(alpha,beta,rho,mu,c_t,k_t,w_t,r_t)
    return np.dot(np.dot(err.T, W), err) 

In [5]:
# Run GMM Estimation
alpha_0,beta_0,rho_0,mu_0 = .5, .5, 0, 1
params_init = np.array([alpha_0,beta_0,rho_0,mu_0])

W_0 = np.eye(4)
gmm_args = (macroseries['c_t'],macroseries['k_t'],macroseries['w_t'],macroseries['r_t'], W_0)

minimizer_kwargs = dict(method="L-BFGS-B", 
                        bounds=((1e-10, 1-1e-10), (1e-10, 1-1e-10), (-1+1e-10, 1-1e-10), (1e-10, None)),
                        args = (gmm_args))
GMM_results = opt.basinhopping(GMM_criterion, params_init, minimizer_kwargs = minimizer_kwargs)

alpha_1,beta_1,rho_1,mu_1 = GMM_results.x
GMM_results

                        fun: 4.047268883151051e-05
 lowest_optimization_result:       fun: 4.047268883151051e-05
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  8.29301316e-04,   1.04744236e+06,  -3.43333524e-01,
         0.00000000e+00])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 125
      nit: 4
   status: 0
  success: True
        x: array([  0.90154807,   0.98997495,   1.        ,  21.38776043])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 9
                       nfev: 13130
                        nit: 100
                          x: array([  0.90154807,   0.98997495,   1.        ,  21.38776043])

In [6]:
GMM_criterion(np.array([0.839707790066,0.989974949529,0.9999999999,14.1403600723]), 
              macroseries['c_t'],macroseries['k_t'],macroseries['w_t'],macroseries['r_t'], W_0)

1.8249758558934258e-05

## Estimating the Brock and Mirman (1972) model by SMM

One nice property of the Brock and Mirman (1972) model is that the household decision has a known analytical solution in which the optimal savings decision $k_{t+1}$ is a function of the productivity shock today and the amount of capital today $k_t$
\begin{align*}
k_{t+1} & = \alpha \beta e^{z_t} k_t^{\alpha} && \text{(11)}
\end{align*}
With this solution, it is straightforward to simulate the data of the Brock and Mirman (1972) model given parameters $\alpha, \beta, \rho, \mu, \sigma$. First, assume that $z_1 = \mu$ and that $k_1 = mean(k_t)$ from the data. These are initial values that will not change across simulations. Next, draw $T= 100$ normally distributed values of $\epsilon_t \sim N(0,\sigma)$. Note that for SMM we have to return to fully specifying the distributional assumptions. Then, you can use equation (5) to calculate the simulated series for $z_t$. Now, you can use the policy function for savings (11) recursively to solve for the entire $k_t$ series. With the entire $k_t$ and $z_t$ simulated series, you can use (3) to solve for the $w_t$ series and (4) to solve for the $r_t$ series. Lastly, you use the budget constraint (2) to solve for the $c_t$ series.

(a) Estimate the five parameters of the Brock and Mirman (1972) model (\alpha,\beta,\rho,\mu,\sigma) described by equations (1) through (5) by SMM. Choose the five parameters to match the following six moments from the 100 periods of empirical data $\{c_t,k_t,w_t,r_t\}_{t=1}^{100}$ in Macroseries.txt: mean($c_t$), mean($k_t$), var($c_t$), var($k_t$), corr($c_t,k_t$), corr($k_t,k_{t+1}$). In your simulations of the model, set $T=100$ and $S=1000$. Start each of your simulations from $k_1 = mean(k_t)$ from the MacroSeries.txt file and $z_1 = \mu$. Input the bounds to be $\alpha,\beta \in [0.01,0.99], \rho \in [-0.99,0.99],\mu \in [-0.5,1],$ and $\sigma \in [0.001,1]$. Also, use the identity matrix as your weighting matrix $W$. Report your solution $\hat{\theta} = (\hat{\alpha},\hat{\beta},\hat{\rho},\hat{\mu}, \hat{\sigma})$ the vector of moment differences at the optimum, and the criterion function value.

In [10]:
def simul_moments(alpha,beta,rho,mu,sigma,k_1,S,T):
    results = np.empty([S,6])
    for s in range(0,S):
        epsilon_t = np.random.normal(loc=0.0, scale=1, size=100)    
        z_t = []
        z_t.append(mu)
        k_t = []
        k_t.append(k_1)
        for t in range(0,T):
            z_t_1 = rho*z_t[t] + (1-rho)*mu + epsilon_t[t]
            z_t.append(z_t_1)
            k_t_1 = alpha*beta*np.exp(z_t[t])*k_t[t]**alpha
            k_t.append(k_t_1)
        z_t = pd.Series(z_t)
        k_t = pd.Series(k_t)
        w_t = (1-alpha)*np.exp(z_t)*k_t**alpha
        r_t = alpha*np.exp(z_t)*k_t**(alpha-1)
        c_t = -k_t.shift(-1) + w_t + r_t*k_t
        results[s,:] = np.array([np.nanmean(c_t),
                                np.nanmean(k_t),
                                np.nanvar(c_t),
                                np.nanvar(k_t),
                                sts.pearsonr(c_t[:-1],k_t[:-1])[0],
                                sts.pearsonr(k_t[:-1],k_t.shift(-1)[:-1])[0]])
    return np.mean(results,axis=0)

def data_moments(c_t,k_t):
    return np.array([np.nanmean(c_t),
                    np.nanmean(k_t),
                    np.nanvar(c_t),
                    np.nanvar(k_t),
                    sts.pearsonr(c_t,k_t)[0],
                    sts.pearsonr(k_t[:-1],k_t.shift(-1)[:-1])[0]])

    
def SMM_criterion(params,*args):
    
    alpha,beta,rho,mu,sigma = params
    c_t,k_t,w_t,r_t,W,S,data_mom = args
    
    T = len(macroseries['k_t'])
    k_1 = np.mean(macroseries['k_t'])
    
    simul_mom = simul_moments(alpha,beta,rho,mu,sigma,k_1,S,T)
    
    err = (simul_mom - data_mom)/data_mom
    
    return np.dot(np.dot(err.T, W), err) 

In [11]:
# Test SimulMoments
print(simul_moments(.5, .5, .5, .5, .5,np.mean(macroseries['k_t']),1000,100))
# Test DataMoments
print(data_moments(macroseries['c_t'],macroseries['k_t']))
# Test Criterion
alpha_0,beta_0,rho_0,mu_0,sigma_0 = .5, .5, .5, .5, .5
print(SMM_criterion(np.array([alpha_0,beta_0,rho_0,mu_0,sigma_0]),
                   macroseries['c_t'],macroseries['k_t'],macroseries['w_t'],macroseries['r_t'], np.eye(6), 1000,
                   data_moments(macroseries['c_t'],macroseries['k_t'])))

[  3.68517918e+01   7.39963207e+04   1.13140638e+05   5.47363782e+11
   9.99000784e-01   9.99000784e-01]
[  1.05213000e+07   7.47240000e+06   5.63027331e+12   2.81408024e+12
   8.77524504e-01   8.76685140e-01]
3.66765893502


In [12]:
# Run SMM Estimation
alpha_0,beta_0,rho_0,mu_0,sigma_0 = .5, .5, .5, .5, .5
params_init = np.array([alpha_0,beta_0,rho_0,mu_0,sigma_0])

W_0 = np.eye(6)
S=1000
data_mom=data_moments(macroseries['c_t'],macroseries['k_t'])
smm_args = (macroseries['c_t'],macroseries['k_t'],macroseries['w_t'],macroseries['r_t'], W_0, S, 
           data_mom)

minimizer_kwargs = dict(method="L-BFGS-B", 
                        bounds=((1e-10, 1-1e-10), (1e-10, 1-1e-10), (-1+1e-10, 1-1e-10), (1e-10, None), (1e-10, None)),
                        args = (smm_args))
SMM_results = opt.basinhopping(SMM_criterion, params_init, minimizer_kwargs = minimizer_kwargs, niter=10)

alpha_2,beta_2,rho_2,mu_2,sigma_2 = SMM_results.x
SMM_results

                        fun: 3.6312969705632341
 lowest_optimization_result:       fun: 3.6312969705632341
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 140431.35791471,   12757.19221585,   74079.96497291,
        103594.14731909,   46423.8578624 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 162
      nit: 3
   status: 0
  success: True
        x: array([  3.20899065e-01,   6.00791346e-01,   7.60296933e-01,
         1.02758945e-07,   1.22605066e-01])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 1
                       nfev: 1482
                        nit: 10
                          x: array([  3.20899065e-01,   6.00791346e-01,   7.60296933e-01,
         1.02758945e-07,   1.22605066e-01])